In [ ]:
%pip install -qU langchain
%pip install -qU huggingface_hub
%pip install nest-asyncio
%pip install unstructured
%pip install chromadb

In [194]:
import os
# Necessary to import OpenAI
with open('assets/openai_api_key', 'r') as f:
    openai_api_key = f.read()
with open('assets/huggingface_api_key', 'r') as f:
    huggingface_api_key = f.read()
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['HUGGINGFACEHUB_API_KEY'] = huggingface_api_key
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader
from langchain.output_parsers import RegexParser
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
import json
import pandas as pd
import re
from langchain.chains import RetrievalQA
import ipywidgets

# OpenAI QA

We use Langchain to create a Chroma database of our papers against which we will query our questions. The answers are provided in the requested structure, making it straightforward to extract information.

Every paper is evaluated separately.

TODO: improve documentation

https://python.langchain.com/docs/modules/chains/additional/question_answering.html

https://python.langchain.com/docs/modules/chains/additional/openai_functions_retrieval_qa


Setting up the parameters:

In [5]:
paper_path = '../data/docs/'
pickle_path = '../data/pickle/'
chroma_path = '../data/chroma/'
MODEL = 'gpt-3.5-turbo'

Creating the vector database:

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Defining the prompt template:

All together?

In [209]:
prompt_template = """We are trying to extract information from a journal article that focuses on bioassays performed on graphene oxide to investigate its effects on and interactions with biological systems and organisms. Our objective is to extract details about the assays conducted and how different measurement groups and conditions contribute to various effects.

Please use the provided journal article as context to fill out the keys in the format defined below. Ensure that the answer to each key is a comma-separated list. Each element in a given key's list should correspond with the respective elements in other keys, maintaining the same length for all lists. Leaving blanks or having lists of different lengths is not acceptable. Repeated elements are allowed, as the paper describes different conditions for each assay, different endpoints for each animal model, and so on. Be as exhaustive as you can, and do not forget to give an answer for every list element (you will respond 'not specified' if you can't find an answer).


Please return the answer to each key as a comma-separated list in such a way that e.g. element 1 of a given key (e.g., diameter measurement units) corresponds with element 1 from a different key (e.g. diameter measurement values) and element 1 from another key (Interaction). This means that it is crucial that all lists must be of the same length -an answer with lists of different lenghts is not acceptable. Each position in the lists represents an individual set of assay conditions, measurements and outcomes, which means that there can be several instances of the same outcome, materials, measurements and conditions.

Format:
---------
question i: [list i of length N]
question j: [list j of length N]
---------


Context:

{context}
---------
Questions: 
{question}

"""



In [210]:
questions = ["Observed interaction or effect",
             "Sample preparation"
             "Bioassays", 
             "Doses", 
             "Doses units", 
             "Coatings", 
             "Organisms/Biological systems", 
             "Shapes", 
             "Diameters measurement units", 
             "Diameters measurement values",
             "Diameters measurement types",
             "Time point values",
             "Time point units", 
             "Passages"]

query = """
    Observed interaction or effect: [List of observed outcomes]
    Sample preparation technique: [List of sample preparation techniques]
    Bioassays: [List of specific bioassays performed]
    Doses: [List of graphene oxide doses]
    Doses units: [List of units for graphene oxide doses]
    Coatings: [List of nanomaterial coating types reported]
    Organisms/Biological systems: [List of organisms or biological systems used in  the assay]
    Shapes: [List of graphene oxide shapes that produced the outcomes]
    Diameters measurement units: [List of units for diameter measurements]
    Diameters measurement values: [List of numeric values for diameter  measurements]
    Diameters measurement types: [List of types of diameter measurements]
    Time point values: [List of time points at which effects were observed]
    Time point units: [List of units for time points]
    Passages: [List of literal excerpts from the text asserting the outcomes]

"""
regex = "(.*)\n".join(questions) + '\n(.*)'


output_parser = RegexParser(
    regex=regex,
    output_keys=questions,
)

doc_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question'],
    output_parser=output_parser,
)

In [211]:
def papers_to_vectors(paper_path):
    #TODO be able to restore persistent chroma
    loader = DirectoryLoader(paper_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    embeddings = OpenAIEmbeddings()
    docsearch = Chroma.from_documents(texts, embeddings,persist_directory=f'../data/chroma/{paper_path}', metadatas=[{"source": str(i)} for i in range(len(texts))])
    docsearch.persist()
    return docsearch
#documents = os.listdir(paper_path)

In [261]:
def get_data(path, runs, query, doc_prompt):
    docsearch = papers_to_vectors(path)
    res = {str(i): {} for i in range(runs)}
    for i in range(runs):
        print(f'Run #{i} ...')
        chain_type_kwargs = {'prompt': doc_prompt}
        qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="maprerank", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs, return_source_documents=True)
        result = qa({'query': query})
        rows = result['result'].split("\n")
        
        for row in rows:
            if 'Answer' not in row and ':' in row:
                row_name, values = row.split(":", 1)
                values = values.split(",")
                res[str(i)][row_name] = values
        res[str(i)]['source_documents'] = result['source_documents']
    res['query'] = result['query']
    return res





In [259]:
path = '../data/docs/test/'
runs = 10
query = query
test_am3 = get_data(path, runs, query, doc_prompt)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Run #0 ...


ValueError: Got unsupported chain type: map-rerank. Should be one of dict_keys(['stuff', 'map_reduce', 'refine', 'map_rerank'])

In [256]:
test_am3

{'0': {'Observed interaction or effect': [' cytotoxicity',
   ' not specified',
   ' adverse effects',
   ' not specified',
   ' not specified',
   ''],
  'Sample preparation technique': [' electrochemically exfoliated',
   ' sonication',
   ' not specified',
   ' not specified',
   ' '],
  'Bioassays': [' MTT assay',
   ' cell viability assay',
   ' hemolysis assay',
   ' not specified',
   ' '],
  'Doses': [' 0.5', ' 0.125', ' 0.25', ' 0.1', ' '],
  'Dose units': [' μg/mL', ' μg/mL', ' μg/mL', ' μg/mL', ' '],
  'Coatings': [' not specified',
   ' not specified',
   ' not specified',
   ' not specified',
   ' '],
  'Organisms/Biological systems': [' E.coli',
   ' human red blood cells',
   ' human embryonic kidney cells',
   ' not specified',
   ' '],
  'Shapes': [' flakes', ' nanoribbons', ' nanosheets', ' not specified', ' '],
  'Diameters measurement units': [' nm', ' nm', ' nm', ' not specified', ' '],
  'Diameters measurement values': [' 5-20',
   ' 10-50',
   ' 20-200',
   ' not

In [255]:

def occurrence_interactions(results, runs):
    seen_interactions = {}
    for iteration in results.keys():
        if type(results[iteration]) == dict:
            values = results[iteration]:
            interaction = values['Observed interaction or effect']:
            if interaction not in seen_interactions.keys():
                seen_interactions[interaction] = {'vals':{}, 'count': 1}
            
            else:
                seen_interactions[interaction]['count'] += 1
            for other_keys in values.keys():
                
    occurences = {i: {'vals': seen_interactions[i]['vals'], 'freq': seen_interactions[i]['count']/runs} for i in seen_interactions.keys()}
    return occurences
seen = occurrence_interactions(test_am3, runs)
seen

{' cytotoxicity': 0.7,
 ' not specified': 0.6,
 ' adverse effects': 0.1,
 '': 0.7,
 ' toxicity': 0.4,
 ' genotoxicity': 0.7,
 ' ': 0.3,
 ' Cytotoxicity': 0.1,
 ' Genotoxicity': 0.1,
 ' Changes in gene expression': 0.1,
 ' Increased antioxidant activity': 0.1,
 ' Increased cancer cell death': 0.1,
 ' Increased macrophage activity': 0.1,
 ' Increased cell uptake': 0.1,
 ' Increased cell proliferation': 0.1,
 ' oxidative stress': 0.5,
 ' ferroptosis': 0.1,
 ' no toxicity': 0.1,
 ' mortality': 0.2,
 ' cell death': 0.1,
 ' apoptosis': 0.3,
 ' DNA damage': 0.1,
 ' inflammation': 0.2,
 ' tissue damage': 0.1,
 ' enzyme activity': 0.1,
 ' cytokine levels': 0.1,
 ' gene expression': 0.2,
 ' protein expression': 0.1,
 ' phenotypic changes': 0.1,
 ' morphological changes': 0.1,
 ' phytotoxic effects': 0.1,
 ' antioxidant activity': 0.1,
 ' inhibition of seed germination': 0.1,
 ' death': 0.1,
 ' adhesion': 0.1,
 ' pro-inflammatory response': 0.1,
 ' no effect': 0.3,
 ' inflammatory response': 0.1,

In [239]:
test_am3

{'0': {'Observed interaction or effect': [' cytotoxicity',
   ' not specified',
   ' adverse effects',
   ' not specified',
   ' not specified',
   ''],
  'Sample preparation technique': [' electrochemically exfoliated',
   ' sonication',
   ' not specified',
   ' not specified',
   ' '],
  'Bioassays': [' MTT assay',
   ' cell viability assay',
   ' hemolysis assay',
   ' not specified',
   ' '],
  'Doses': [' 0.5', ' 0.125', ' 0.25', ' 0.1', ' '],
  'Dose units': [' μg/mL', ' μg/mL', ' μg/mL', ' μg/mL', ' '],
  'Coatings': [' not specified',
   ' not specified',
   ' not specified',
   ' not specified',
   ' '],
  'Organisms/Biological systems': [' E.coli',
   ' human red blood cells',
   ' human embryonic kidney cells',
   ' not specified',
   ' '],
  'Shapes': [' flakes', ' nanoribbons', ' nanosheets', ' not specified', ' '],
  'Diameters measurement units': [' nm', ' nm', ' nm', ' not specified', ' '],
  'Diameters measurement values': [' 5-20',
   ' 10-50',
   ' 20-200',
   ' not

In [170]:
rows = result['result'].split("\n")
new_table = []
row_names = []
for row in rows:
    if ':' in row:
        row_name = row.split(":")[0]
        values = row.split(":")[1]
        row_names.append(row_name)
        values = values.split(",")
        new_table.append(values)
df = pd.DataFrame(new_table, index = row_names).drop('Answer')
df


,0,1,2,3,4,5,6,7,8,9,10
Observed interaction or effect,cell death,cytotoxicity,inflammation,oxidative stress,DNA damage,apoptosis,genotoxicity,increased intracellular ROS,decreased cell viability,altered cellular morphology,
Sample preparation technique,sonication,centrifugation,,None,None,None,None,None,None,None,None
Bioassays,MTT assay,comet assay,flow cytometry,,None,None,None,None,None,None,None
Doses,0.5,1,10,100,,None,None,None,None,None,None
Doses units,μg/ml,,None,None,None,None,None,None,None,None,None
Coatings,not specified,,None,None,None,None,None,None,None,None,None
Organisms/Biological systems,human lung fibroblasts,human peripheral blood mononuclear cells,human macrophages,,None,None,None,None,None,None,None
Shapes,nanosheets,,None,None,None,None,None,None,None,None,None
Diameters measurement units,nm,,None,None,None,None,None,None,None,None,None
Diameters measurement values,2.1,5.4,8.0,,None,None,None,None,None,None,None


In [171]:
result['result']

'Answer: \n\nObserved interaction or effect: cell death, cytotoxicity, inflammation, oxidative stress, DNA damage, apoptosis, genotoxicity, increased intracellular ROS, decreased cell viability, altered cellular morphology,\nSample preparation technique: sonication, centrifugation,\nBioassays: MTT assay, comet assay, flow cytometry,\nDoses: 0.5, 1, 10, 100,\nDoses units: μg/ml,\nCoatings: not specified,\nOrganisms/Biological systems: human lung fibroblasts, human peripheral blood mononuclear cells, human macrophages,\nShapes: nanosheets,\nDiameters measurement units: nm,\nDiameters measurement values: 2.1, 5.4, 8.0,\nDiameters measurement types: average,\nTime point values: 24, 48, 72,\nTime point units: hours,\nPassages: "The results showed that GO nanosheets were capable of inducing cell death, cytotoxicity, inflammation, and oxidative stress in human lung fibroblasts. GO induced a significant increase in intracellular ROS levels and decreased cell viability in a dose-dependent'

'Answer: Observed interaction or effect (each observed outcome): no effect, mortality, altered behavior, biochemical changes, Sample preparation technique: sonication, centrifugation, Bioassays (What specific bioassays were performed?): mortality assessment, behavioral assays, biochemical assays, Doses (What were the graphene oxide doses?): 0.5, 5, 25, Doses units (Units of the graphene oxide doses?): µg/mL, Coatings (What types of nanomaterial coating are reported?): uncoated, Organisms/Biological systems (What organisms or biological systems were the assay performed on (include strains, if applicable)?): Daphnia magna, Danio rerio, E. coli, Shapes (What is the shape of the graphene oxide that produced this outcome?): nanosheets, Diameters measurement units (nanometers, etc): nanometers, Diameters measurement values (the numeric value of the diameter measurement): 10, 20, 30, Diameters measurement types (hydrodynamic diameter, etc.): hydrodynamic diameter, Time point values (the time 